In [2]:
import gzip
import re
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from dataclasses import dataclass
from typing import Iterator, List
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
@dataclass
class Text:
    label: str
    title: str
    text: str

def read_texts(fn: str="data/news.txt.gz") -> Iterator[Text]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield Text(*line.strip().split("\t"))

def tokenize_text(text: str) -> List[str]:
    text = text.lower()
    words = re.findall(r'\b\w+\b', text.lower())
    return words

def normalize_text(text: str) -> str:
    return ' '.join(tokenize_text(text))


In [4]:
# Функция для усреднения векторов
def average_word_vectors(words, model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,), dtype="float32")
    nwords = 0

    for word in words:
        if word in vocabulary: 
            nwords += 1
            feature_vector = np.add(feature_vector, model.wv[word])

    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
    return feature_vector

def averaged_word_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index_to_key)
    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features) for tokenized_sentence in corpus]
    return np.array(features)


In [5]:
def tfidf_weighted_averaged_word_vectorizer(corpus, tfidf_vectors, tfidf_vocabulary, model, num_features):
    doc_word_vector = np.zeros((len(corpus), num_features), dtype="float32")
    for idx, tokenized_sentence in enumerate(corpus):
        for word in tokenized_sentence:
            if word in model.wv.key_to_index and word in tfidf_vocabulary:
                word_vector = model.wv[word]
                tfidf_weight = tfidf_vectors[idx][tfidf_vocabulary[word]]
                doc_word_vector[idx] = np.add(doc_word_vector[idx], word_vector * tfidf_weight)
    return np.divide(doc_word_vector, len(corpus))


In [22]:
# Чтение данных
texts = list(read_texts("data/news.txt.gz"))
sentences = [tokenize_text(text.text) for text in texts]

# Обучение Word2Vec
w2v = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)


In [23]:
# Усреднение векторов для каждого документа
w2v_feature_array = averaged_word_vectorizer(corpus=sentences, model=w2v, num_features=100)

# Разделение на обучающую и тестовую выборки
y = [text.label for text in texts]

X_train, X_test, y_train, y_test = train_test_split(w2v_feature_array, y, test_size=0.3, random_state=42)

# Обучение модели SVM на обычных векторах
svc = SVC()
svc.fit(X_train, y_train)
accuracy = svc.score(X_test, y_test)
print(f"Точность классификации на обычных векторах: {accuracy}")


Точность классификации на обычных векторах: 0.66


Первый альтернативный способ представления документов: TF-IDF взвешенные векторы

In [24]:
vectorizer = TfidfVectorizer(max_df=0.2, min_df=10)
X_tfidf = vectorizer.fit_transform([normalize_text(text.text) for text in texts]).toarray()

In [25]:
tfidf_feature_array = tfidf_weighted_averaged_word_vectorizer(corpus=sentences, tfidf_vectors=X_tfidf, tfidf_vocabulary=vectorizer.vocabulary_, model=w2v, num_features=100)

# Разделение на обучающую и тестовую выборки для TF-IDF взвешенных векторов
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(tfidf_feature_array, y, test_size=0.3, random_state=42)

# Обучение модели SVM на TF-IDF взвешенных векторах
svc_tfidf = SVC()
svc_tfidf.fit(X_train_tfidf, y_train_tfidf)
accuracy_tfidf = svc_tfidf.score(X_test_tfidf, y_test_tfidf)
print(f"Точность классификации на TF-IDF взвешенных векторах: {accuracy_tfidf}")

Точность классификации на TF-IDF взвешенных векторах: 0.6816666666666666


Второй альтернативный метод: Doc2Vec

In [26]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Подготовка данных для Doc2Vec
tagged_data = [TaggedDocument(words=tokenize_text(text.text), tags=[i]) for i, text in enumerate(texts)]

# Обучение модели Doc2Vec
doc2vec_model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=40)
doc2vec_model.build_vocab(tagged_data)
doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

# Получение векторов для каждого документа
doc_vectors = np.array([doc2vec_model.dv[i] for i in range(len(tagged_data))])

# Разделение на обучающую и тестовую выборки
X_train_doc2vec, X_test_doc2vec, y_train_doc2vec, y_test_doc2vec = train_test_split(doc_vectors, y, test_size=0.3, random_state=42)

# Обучение и тестирование SVM
svc_doc2vec = SVC()
svc_doc2vec.fit(X_train_doc2vec, y_train_doc2vec)
accuracy_doc2vec = svc_doc2vec.score(X_test_doc2vec, y_test_doc2vec)
print(f"Точность классификации с Doc2Vec: {accuracy_doc2vec}")


Точность классификации с Doc2Vec: 0.822


Третий альтернативный способ: n-gramms

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.decomposition import PCA

# Подготовка текстов и их предобработка
texts = list(read_texts("data/news.txt.gz"))
processed_texts = [normalize_text(text.text) for text in texts]

# Создание n-грамм
ngram_vectorizer = TfidfVectorizer(max_df=0.2, min_df=10, ngram_range=(1, 2))  # Использование униграмм и биграмм
X_ngram = ngram_vectorizer.fit_transform(processed_texts).toarray()

# Разделение на обучающую и тестовую выборки
y = [text.label for text in texts]
X_train_ngram, X_test_ngram, y_train_ngram, y_test_ngram = train_test_split(X_ngram, y, test_size=0.3, random_state=42)

# Применение PCA для уменьшения размерности
pca = PCA(n_components=0.95)  # сохраняем 95% вариативности
X_train_ngram_pca = pca.fit_transform(X_train_ngram)
X_test_ngram_pca = pca.transform(X_test_ngram)

# Обучение и тестирование SVM
svc_ngram = SVC()
svc_ngram.fit(X_train_ngram_pca, y_train_ngram)
accuracy_ngram = svc_ngram.score(X_test_ngram_pca, y_test_ngram)
print(f"Точность классификации с использованием n-грамм и PCA: {accuracy_ngram}")


Точность классификации с использованием n-грамм и PCA: 0.845
